## Recurrent Neural Networks

In this chapter, we
are going to discuss recurrent neural networks (RNN), a class of nets that can predict
the future (well, up to a point, of course). They can analyze time series data such as
stock prices, and tell you when to buy or sell. In autonomous driving systems, they
can anticipate car trajectories and help avoid accidents. More generally, they can work
on sequences of arbitrary lengths, rather than on fixed-sized inputs like all the nets we
have discussed so far. For example, they can take sentences, documents, or audio
samples as input, making them extremely useful for natural language processing
(NLP) systems such as automatic translation, speech-to-text, or sentiment analysis
(e.g., reading movie reviews and extracting the rater’s feeling about the movie).
Moreover, RNNs’ ability to anticipate also makes them capable of surprising creativity.
You can ask them to predict which are the most likely next notes in a melody, then
randomly pick one of these notes and play it. Then ask the net for the next most likely
notes, play it, and repeat the process again and again. Before you know it, your net
will compose a melody such as the one produced by Google’s Magenta project. Similarly,
RNNs can generate sentences, image captions, and much more. The result is not
exactly Shakespeare or Mozart yet, but who knows what they will produce a few years
from now?
In this chapter, we will look at the fundamental concepts underlying RNNs, the main
problem they face (namely, vanishing/exploding gradients, discussed in Chapter 11),
and the solutions widely used to fight it: LSTM and GRU cells. Along the way, as
always, we will show how to implement RNNs using TensorFlow. Finally, we will take
a look at the architecture of a machine translation system.

In [2]:
import tensorflow as tf

## Recurrent Neurons

A recurrent neural network looks very much like a feedforward neural network, except it also has connections pointing backward. the simplest possible RNN, composed of just one neuron receiving inputs, producing an output, and sending that output back to itself. We can represent this tiny network against the time axis (unrolling the network through time)

![title](img/RNN_single.png)

You can easily create a layer of recurrent neurons. At each time step t, every neuron
receives both the input vector x(t) and the output vector from the previous time step
y(t–1).
Each recurrent neuron has two sets of weights: one for the inputs x(t) and the other for
the outputs of the previous time step, y(t–1). Let’s call these weight vectors wx and wy.

![title](img/RNN_single_math.png)

### Memory Cells
Since the output of a recurrent neuron at time step t is a function of all the inputs
from previous time steps, you could say it has a form of memory. A part of a neural
network that preserves some state across time steps is called a memory cell (or simply
a cell). A single recurrent neuron, or a layer of recurrent neurons, is a very basic cell,
but later in this chapter we will look at some more complex and powerful types of
cells. In general a cell’s state at time step t, denoted h(t) (the “h” stands for “hidden”), is a
function of some inputs at that time step and its state at the previous time step: h(t) =
f(h(t–1), x(t)). Its output at time step t, denoted y(t), is also a function of the previous
state and the current inputs. In the case of the basic cells we have discussed so far, the
output is simply equal to the state, but in more complex cells this is not always the
case, as shown here:

![title](img/RNN_memory_cell.png)

### Input and Output Sequences
An RNN can simultaneously take a sequence of inputs and produce a sequence of
outputs (see Figure 14-4, top-left network). For example, this type of network is useful
for predicting time series such as stock prices: you feed it the prices over the last N
days, and it must output the prices shifted by one day into the future (i.e., from N – 1
days ago to tomorrow).

Alternatively, you could feed the network a sequence of inputs, and ignore all outputs
except for the last one (see the top-right network). In other words, this is a sequenceto-
vector network.For intance, text corresponding to a movie review, and the network would output a sentiment score (e.g.,
from –1 [hate] to +1 [love]).

Conversely, you could feed the network a single input at the first time step (and zeros
for all other time steps), and let it output a sequence (see the bottom-left network).
This is a vector-to-sequence network. For example, the input could be an image, and
the output could be a caption for that image.

Lastly, you could have a sequence-to-vector network, called an encoder, followed by a
vector-to-sequence network, called a decoder (see the bottom-right network). For
example, this can be used for translating a sentence from one language to another.
You would feed the network a sentence in one language, the encoder would convert
this sentence into a single vector representation, and then the decoder would decode
this vector into a sentence in another language. This two-step model, called an
Encoder–Decoder, works much better than trying to translate on the fly with a single
sequence-to-sequence RNN (like the one represented on the top left), since the last
words of a sentence can affect the first words of the translation, so you need to wait
until you have heard the whole sentence before translating it.

![title](img/RNN_types.png)

Here is a 1 layer RNN without using any of the RNN tensorflow operations:

![title](img/RNN_single_5.png)

In [3]:
n_inputs = 3
n_neurons = 5
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)
init = tf.global_variables_initializer()

In [5]:
import numpy as np
# Mini-batch: instance 0,instance 1,instance 2,instance 3
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [6]:
print(Y0_val)
print('--------------------------')
print(Y1_val)

[[ 0.98625517 -0.98694319 -0.53638977  0.85512584  0.41621998]
 [ 0.99999219 -1.         -0.81734431  0.99998057  0.99985391]
 [ 1.         -1.         -0.93521559  1.          1.        ]
 [ 0.97881645 -1.         -0.99998969  0.99999964  1.        ]]
--------------------------
[[ 1.         -1.         -0.98802906  1.          1.        ]
 [ 0.72207105 -0.42682648 -0.84667838 -0.8303107   0.99592489]
 [ 0.99998808 -1.         -0.90301752  0.99999446  1.        ]
 [ 0.98160332 -0.9999997  -0.66550261  0.94867563  0.99999815]]


## TF RNN FUNCTIONS

#### Static Unrolling Through Time

The static_rnn() function creates an unrolled RNN network by chaining cells. The
following code creates the exact same model as the previous one:

In [7]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)
Y0, Y1 = output_seqs